# HydroGeoSines
## HALS
Short tutorial on how to apply the HALS method to both regularly and irregularly sampled data.

### Import HGS
Currently, the HydroGeoSines is not fully implemented as an installable package. Instead. we have to move to the parent directory, to import the package.

In [1]:
import os
os.chdir("../../")
print("Current Working Directory " , os.getcwd())

# Load the HGS package
import hydrogeosines as hgs

Current Working Directory  /media/daniel/SharedData/Workspaces/GitHub/HydroGeoSines


In [2]:
# and other packages used in this tutorial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Create a Site object
example_site = hgs.Site('example', geoloc=[141.762065, -31.065781, 160])
print(example_site)

### Load Data
### The groundwater head records

In [4]:
# Load all our data attributed to the Site
example_site.import_csv('tests/data/notebook/GW_record.csv', 
                        input_category=["GW"]*3, 
                        utc_offset=10, 
                        unit=["m"]*3,
                        loc_names = ["Loc_A","Loc_B"], 
                        header = None,
                        check_duplicates=True) 

A new time series was added ...
No duplicate entries were found.


### The barometric pressure records

In [5]:
example_site.import_csv('tests/data/notebook/BP_record.csv', 
                        input_category="BP", 
                        utc_offset=10, 
                        unit="m", 
                        loc_names = "Baro",
                        header = None,
                        how="add", check_duplicates=True) 

A new time series was added ...
No duplicate entries were found.


### HALS
#### With regularly sampled data
First we need to regularly sample our groundwater data and align it with te BP records. The RegularAndAlign method does just that. However, the original data in the processing object is not yet updated, as the regular data has its own attribute:

In [6]:
process_HALS_rs = hgs.Processing(example_site).RegularAndAligned(inter_max=5000, part_min=20,inter_max_total=40)
process_HALS_rs.site.data = process_HALS_rs.data_regular

4.73 % of the 'GW' data at 'Loc_A_all' was interpolated due to gaps < 5000s!
5.08 % of the 'GW' data at 'Loc_B_all' was interpolated due to gaps < 5000s!
Data of the category 'GW' is regularly sampled now!

Start iteration No. 1 ...

----- Loc_A_1 -----
BP record resampled to 1 sample per 300s.

Processing BP gaps ...
0.02 % of the 'BP' data at 'Baro_all' was interpolated due to gaps < 5000s!
... record gaps between 2001-02-27 16:30 and 2001-03-01 07:15 too large for interpolation!

Processing GW gaps ...
... dropping GW and BP entries for which BP record gaps are too big.
0.00 % of the 'GW' data at 'Loc_A_1' was interpolated due to gaps < 5000s!

----- Loc_B_1 -----
BP record resampled to 1 sample per 300s.

Processing BP gaps ...
6.10 % of the 'BP' data at 'Baro_all' was interpolated due to gaps < 5000s!
... record gaps between 2001-02-27 16:30 and 2001-03-01 07:15 too large for interpolation!

Processing GW gaps ...
... dropping GW and BP entries for which BP record gaps are too big

In [10]:
results_HALS_rs = process_HALS_rs.hals("Loc_B")

-------------------------------------------------
Method: hals
-------------------------------------------------
> Calculating HALS for location: Loc_B
Data category: GW
>> Condition number: 102
>> Error variance: 0.000031
>> DC component: 0.000007
Data category: BP
>> Condition number: 89
>> Error variance: 0.000035
>> DC component: 0.000097
-------------------------------------------------
> Calculating HALS for location: Loc_B
Data category: GW
>> Condition number: 510
>> Error variance: 0.000014
>> DC component: 0.000018
Data category: BP
>> Condition number: 226
>> Error variance: 0.000157
>> DC component: -0.000030


#### With irregularly sampled data and BP records that are not aligned with the GW data

In [11]:
# Running HALS with irregularly sampled data
results_HALS_irs = hgs.Processing(example_site).hals(loc="Loc_B")

-------------------------------------------------
Method: hals
-------------------------------------------------
> Calculating HALS for location: Loc_B
Data category: BP
>> Condition number: 33
>> Error variance: 0.000230
>> DC component: -0.000031
Data category: GW
>> Condition number: 23
>> Error variance: 0.000025
>> DC component: -0.000008


In [44]:
# irregular sampling results
BP_irs = results_HALS_irs["hals"][("Loc_B","all","BP")][0]["error_var"]
GW_irs = results_HALS_irs["hals"][("Loc_B","all","GW")][0]["error_var"]

# regular sampling results
BP_1_rs = results_HALS_rs["hals"][("Loc_B","1","BP")][0]["error_var"]
GW_1_rs = results_HALS_rs["hals"][("Loc_B","1","GW")][0]["error_var"]

BP_2_rs = results_HALS_rs["hals"][("Loc_B","2","BP")][0]["error_var"]
GW_2_rs = results_HALS_rs["hals"][("Loc_B","2","GW")][0]["error_var"]

print("\nThe error variance of the irregularly sampled and non-aligned BP records at Loc_B 1 and 2 is {:.2f}% and {:.2f}% higher, respectively, compared to the regularly sampled data.".format((BP_irs-BP_1_rs)/BP_1_rs,(BP_irs-BP_2_rs)/BP_2_rs))

print("\nThe error variance of the irregularly sampled GW records at Loc_B 1 and 2 is {:.2f}% and {:.2f}% higher, respectively, compared to the regularly sampled data.".format((GW_irs-GW_1_rs)/GW_1_rs,(GW_irs-GW_2_rs)/GW_2_rs))


The error variance of the irregularly sampled and non-aligned BP records at Loc_B 1 and 2 is 5.65% and 0.47% higher, respectively, compared to the regularly sampled data.

The error variance of the irregularly sampled GW records at Loc_B 1 and 2 is -0.20% and 0.76% higher, respectively, compared to the regularly sampled data.
